<a href="https://colab.research.google.com/github/IrynaTkachenko/ai-experience/blob/main/2021_05_25_Spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Oснови Spark

Для запуску в середовищі Google Colab запустимо наступний код

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget https://apache.volia.net/spark/spark-3.1.1/spark-3.1.1-bin-hadoop2.7.tgz

--2021-05-26 11:47:26--  https://apache.volia.net/spark/spark-3.1.1/spark-3.1.1-bin-hadoop2.7.tgz
Resolving apache.volia.net (apache.volia.net)... 82.144.192.7
Connecting to apache.volia.net (apache.volia.net)|82.144.192.7|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 224374704 (214M) [application/x-gzip]
Saving to: ‘spark-3.1.1-bin-hadoop2.7.tgz.1’

spark-3.1.1-bin-had 100%[===================>] 213.98M  17.6MB/s    in 14s     

2021-05-26 11:47:41 (15.5 MB/s) - ‘spark-3.1.1-bin-hadoop2.7.tgz.1’ saved [224374704/224374704]



In [2]:
!tar xf '/content/spark-3.1.1-bin-hadoop2.7.tgz'
!pip install -q findspark

Spark та Java у Colab встановлено.
Встановимо шляхи до середовища, які дозволятимуть запускати Pyspark. 


In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop2.7"


Запустимо локальну сесію Spark.

In [4]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

Вітаємо! Colab вже працює з Pyspark.

#Spark Dataframe

In [5]:
valuesA = [('Pirate',1),('Monkey',2),('Ninja',3),('Spaghetti',4)]
TableA = spark.createDataFrame(valuesA,['name','id'])
 
valuesB = [('Rutabaga',1),('Pirate',2),('Ninja',3),('Darth Vader',4)]
TableB = spark.createDataFrame(valuesB,['name','id'])
 
TableA.show()
TableB.show()

ta = TableA.alias('ta')
tb = TableB.alias('tb')

+---------+---+
|     name| id|
+---------+---+
|   Pirate|  1|
|   Monkey|  2|
|    Ninja|  3|
|Spaghetti|  4|
+---------+---+

+-----------+---+
|       name| id|
+-----------+---+
|   Rutabaga|  1|
|     Pirate|  2|
|      Ninja|  3|
|Darth Vader|  4|
+-----------+---+



#Spark SQL

In [6]:
inner_join = ta.join(tb, ta.name == tb.name)
inner_join.show()

left_join = ta.join(tb, ta.name == tb.name,how='left') 
left_join.show()

right_join = ta.join(tb, ta.name == tb.name,how='right')
right_join.show()

full_outer_join = ta.join(tb, ta.name == tb.name,how='full')
full_outer_join.show()


+------+---+------+---+
|  name| id|  name| id|
+------+---+------+---+
| Ninja|  3| Ninja|  3|
|Pirate|  1|Pirate|  2|
+------+---+------+---+

+---------+---+------+----+
|     name| id|  name|  id|
+---------+---+------+----+
|Spaghetti|  4|  null|null|
|    Ninja|  3| Ninja|   3|
|   Pirate|  1|Pirate|   2|
|   Monkey|  2|  null|null|
+---------+---+------+----+

+------+----+-----------+---+
|  name|  id|       name| id|
+------+----+-----------+---+
|  null|null|   Rutabaga|  1|
| Ninja|   3|      Ninja|  3|
|Pirate|   1|     Pirate|  2|
|  null|null|Darth Vader|  4|
+------+----+-----------+---+

+---------+----+-----------+----+
|     name|  id|       name|  id|
+---------+----+-----------+----+
|     null|null|   Rutabaga|   1|
|Spaghetti|   4|       null|null|
|    Ninja|   3|      Ninja|   3|
|   Pirate|   1|     Pirate|   2|
|   Monkey|   2|       null|null|
|     null|null|Darth Vader|   4|
+---------+----+-----------+----+



# Spark ML. Модель лінійної регресії


Модель лінійної регресії є одним із найдавніших та широко застосовуваних підходів до машинного навчання, який передбачає взаємозв'язок між залежними та незалежними змінними. 

Лінійна регресія складається з лінії, що найкраще підходить, через розрізнені точки на графіку, а лінія, що найкраще підходить, відома як лінія регресії.

Мета цієї вправи прогнозувати ціни на житло за заданими ознаками. Давайте спрогнозуємо ціни набору даних про житло в Бостоні, розглядаючи MEDV як вихідну змінну, а всі інші змінні [текст посилання](https://)як вхідні.

In [7]:
!wget https://raw.githubusercontent.com/asifahmed90/pyspark-ML-in-Colab/master/BostonHousing.csv


--2021-05-26 11:48:07--  https://raw.githubusercontent.com/asifahmed90/pyspark-ML-in-Colab/master/BostonHousing.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 35735 (35K) [text/plain]
Saving to: ‘BostonHousing.csv.1’

BostonHousing.csv.1 100%[===================>]  34.90K  --.-KB/s    in 0.002s  

2021-05-26 11:48:07 (22.5 MB/s) - ‘BostonHousing.csv.1’ saved [35735/35735]



Перевіряємо наявність файлів

In [8]:
!ls

BostonHousing.csv    spark-3.1.1-bin-hadoop2.7	      testsparkjob.py
BostonHousing.csv.1  spark-3.1.1-bin-hadoop2.7.tgz
sample_data	     spark-3.1.1-bin-hadoop2.7.tgz.1


Тепер, коли ми завантажили набір даних, ми можемо розпочати аналіз.
Для нашої моделі лінійної регресії нам потрібно імпортувати два модулі з Pyspark, тобто Vector Assembler та Linear Regression. Vector Assembler - це трансформатор, який збирає всі функції в один вектор із декількох стовпців, що містять тип float/double. Ми могли б використовувати StringIndexer, якщо будь-який із наших стовпців містить значення рядків для перетворення його в числові значення. На щастя, набір даних BostonHousing містить лише подвійні значення, тому наразі нам не потрібно турбуватися про StringIndexer.

In [9]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import StructType, IntegerType, DateType, StringType, FloatType

#'crim', 'zn', 'indus', 'chas', 'nox', 'rm', 'age', 'dis', 'rad', 'tax', 'ptratio', 'b', 'lstat'
schema = StructType([
    StructField("crim", FloatType()),
    StructField("zn", FloatType()),
    StructField("indus", FloatType()),
    StructField("chas", FloatType()),
    StructField("nox", FloatType()),
    StructField("rm", FloatType()),
    StructField("age", FloatType()),
    StructField("dis", FloatType()),
    StructField("rad", FloatType()),
    StructField("tax", FloatType()),
    StructField("ptratio", FloatType()),
    StructField("b", FloatType()),
    StructField("lstat", FloatType()),
    StructField("medv", FloatType()),
])

dataset = spark.read.csv('BostonHousing.csv', schema=schema, header =True, 
                         ignoreLeadingWhiteSpace=True, ignoreTrailingWhiteSpace=True)

In [10]:
# Або
dataset = spark.read.csv('BostonHousing.csv', inferSchema=True, header =True, 
                         ignoreLeadingWhiteSpace=True, ignoreTrailingWhiteSpace=True)

In [11]:
dataset.head(10)

[Row(crim=0.00632, zn=18.0, indus=2.31, chas=0, nox=0.538, rm=6.575, age=65.2, dis=4.09, rad=1, tax=296, ptratio=15.3, b=396.9, lstat=4.98, medv=24.0),
 Row(crim=0.02731, zn=0.0, indus=7.07, chas=0, nox=0.469, rm=6.421, age=78.9, dis=4.9671, rad=2, tax=242, ptratio=17.8, b=396.9, lstat=9.14, medv=21.6),
 Row(crim=0.02729, zn=0.0, indus=7.07, chas=0, nox=0.469, rm=7.185, age=61.1, dis=4.9671, rad=2, tax=242, ptratio=17.8, b=392.83, lstat=4.03, medv=34.7),
 Row(crim=0.03237, zn=0.0, indus=2.18, chas=0, nox=0.458, rm=6.998, age=45.8, dis=6.0622, rad=3, tax=222, ptratio=18.7, b=394.63, lstat=2.94, medv=33.4),
 Row(crim=0.06905, zn=0.0, indus=2.18, chas=0, nox=0.458, rm=7.147, age=54.2, dis=6.0622, rad=3, tax=222, ptratio=18.7, b=396.9, lstat=5.33, medv=36.2),
 Row(crim=0.02985, zn=0.0, indus=2.18, chas=0, nox=0.458, rm=6.43, age=58.7, dis=6.0622, rad=3, tax=222, ptratio=18.7, b=394.12, lstat=5.21, medv=28.7),
 Row(crim=0.08829, zn=12.5, indus=7.87, chas=0, nox=0.524, rm=6.012, age=66.6, di

Зверніть увагу, що ми використовували InferSchema всередині модуля read.csv. InferSchema дозволяє нам автоматично виводити різні типи даних для кожного стовпця.

Давайте роздрукуємося в наборі даних, щоб побачити типи даних кожного стовпця:

In [12]:
dataset.printSchema()

root
 |-- crim: double (nullable = true)
 |-- zn: double (nullable = true)
 |-- indus: double (nullable = true)
 |-- chas: integer (nullable = true)
 |-- nox: double (nullable = true)
 |-- rm: double (nullable = true)
 |-- age: double (nullable = true)
 |-- dis: double (nullable = true)
 |-- rad: integer (nullable = true)
 |-- tax: integer (nullable = true)
 |-- ptratio: double (nullable = true)
 |-- b: double (nullable = true)
 |-- lstat: double (nullable = true)
 |-- medv: double (nullable = true)



Наступним кроком є перетворення всіх функцій з різних стовпців в один стовпець і давайте назвемо цей новий векторний стовпець як 'Attributes' у outputCol.

In [13]:
#Input all the features in one vector column
assembler = VectorAssembler(inputCols=['crim', 'zn', 'indus', 'chas', 'nox', 'rm', 'age', 'dis', 'rad', 'tax', 'ptratio', 'b', 'lstat'], outputCol = 'Attributes')
output = assembler.transform(dataset)
#Input vs Output
finalized_data = output.select("Attributes","medv")

#print(finalized_data)
finalized_data.show()

+--------------------+----+
|          Attributes|medv|
+--------------------+----+
|[0.00632,18.0,2.3...|24.0|
|[0.02731,0.0,7.07...|21.6|
|[0.02729,0.0,7.07...|34.7|
|[0.03237,0.0,2.18...|33.4|
|[0.06905,0.0,2.18...|36.2|
|[0.02985,0.0,2.18...|28.7|
|[0.08829,12.5,7.8...|22.9|
|[0.14455,12.5,7.8...|27.1|
|[0.21124,12.5,7.8...|16.5|
|[0.17004,12.5,7.8...|18.9|
|[0.22489,12.5,7.8...|15.0|
|[0.11747,12.5,7.8...|18.9|
|[0.09378,12.5,7.8...|21.7|
|[0.62976,0.0,8.14...|20.4|
|[0.63796,0.0,8.14...|18.2|
|[0.62739,0.0,8.14...|19.9|
|[1.05393,0.0,8.14...|23.1|
|[0.7842,0.0,8.14,...|17.5|
|[0.80271,0.0,8.14...|20.2|
|[0.7258,0.0,8.14,...|18.2|
+--------------------+----+
only showing top 20 rows



Тут "Attributes" містяться у вхідних функціях з усіх стовпців, а "medv" - цільовий стовпець.
Далі слід розділити дані навчання та тестування відповідно до нашого набору даних (у цьому випадку 0,8 та 0,2).

In [14]:
#Split training and testing data
train_data,test_data = finalized_data.randomSplit([0.8,0.2])


regressor = LinearRegression(featuresCol = 'Attributes', labelCol = 'medv')

#Learn to fit the model from training set
regressor = regressor.fit(train_data)

#To predict the prices on testing set
pred = regressor.evaluate(test_data)

#Predict the model
pred.predictions.show()

+--------------------+----+------------------+
|          Attributes|medv|        prediction|
+--------------------+----+------------------+
|[0.01301,35.0,1.5...|32.7|29.883108263387108|
|[0.01709,90.0,2.0...|30.1|24.923785962928225|
|[0.01778,95.0,1.4...|32.9|30.603828392399723|
|[0.01951,17.5,1.3...|33.0|22.640768692576714|
|[0.02729,0.0,7.07...|34.7|30.642635886097388|
|[0.03049,55.0,3.7...|31.2|28.663917224354197|
|[0.03427,0.0,5.19...|19.5| 20.33525190533025|
|[0.03584,80.0,3.3...|23.5|30.670897995407657|
|[0.03615,80.0,4.9...|27.9| 32.66282048539621|
|[0.04301,80.0,1.9...|18.2|14.041332457416878|
|[0.04337,21.0,5.6...|20.5|23.706177649907012|
|[0.0459,52.5,5.32...|22.3|27.193972377903116|
|[0.0536,21.0,5.64...|25.0|27.640560167868337|
|[0.05425,0.0,4.05...|24.6|29.536830438792215|
|[0.05479,33.0,2.1...|28.4|31.489242242531187|
|[0.05561,70.0,2.2...|29.0|31.764503591806907|
|[0.06127,40.0,6.4...|33.1| 34.99846973039373|
|[0.06263,0.0,11.9...|22.4| 23.78437117684898|
|[0.06417,0.0

Ми також можемо надрукувати коефіцієнт і перехоплення моделі регресії, використовуючи таку команду:

In [15]:
#coefficient of the regression model
coeff = regressor.coefficients

#X and Y intercept
intr = regressor.intercept

print ("The coefficient of the model is : %a" %coeff)
print ("The Intercept of the model is : %f" %intr)


The coefficient of the model is : DenseVector([-0.1443, 0.0522, 0.0175, 2.3272, -18.5393, 3.6335, -0.0002, -1.6747, 0.3657, -0.0138, -0.8988, 0.0087, -0.5934])
The Intercept of the model is : 39.020432


In [16]:
from pyspark.ml.evaluation import RegressionEvaluator
eval = RegressionEvaluator(labelCol="medv", predictionCol="prediction", metricName="rmse")

# Root Mean Square Error
rmse = eval.evaluate(pred.predictions)
print("RMSE: %.3f" % rmse)

# Mean Square Error
mse = eval.evaluate(pred.predictions, {eval.metricName: "mse"})
print("MSE: %.3f" % mse)

# Mean Absolute Error
mae = eval.evaluate(pred.predictions, {eval.metricName: "mae"})
print("MAE: %.3f" % mae)

# r2 - coefficient of determination
r2 = eval.evaluate(pred.predictions, {eval.metricName: "r2"})
print("r2: %.3f" %r2)



RMSE: 4.148
MSE: 17.204
MAE: 3.116
r2: 0.750


#Ще один приклад лінійної регрессії

[Тут](https://www.datacamp.com/community/tutorials/apache-spark-tutorial-machine-learning?utm_source=adwords_ppc&utm_campaignid=1455363063&utm_adgroupid=65083631748&utm_device=c&utm_keyword=&utm_matchtype=b&utm_network=g&utm_adpostion=&utm_creative=278443377095&utm_targetid=dsa-473406582395&utm_loc_interest_ms=&utm_loc_physical_ms=9061015&gclid=CjwKCAjw47eFBhA9EiwAy8kzNGbswRR2ftFQ3G2QRV4sJaXG3gYumCFj3bHvXeHynV-rbUC590Z0VxoCGOcQAvD_BwE)

#Створення Spark служби

Згенеруємо службу та запустимо її

In [17]:
sparkjob =  '''
import sys
 
from pyspark import SparkContext, SparkConf
 
if __name__ == "__main__":
 
  # create Spark context with Spark configuration
  conf = SparkConf().setAppName("Read Text to RDD - Python")
  sc = SparkContext(conf=conf)
 
  # read input text file to RDD
  numbers = sc.parallelize([1,7,8,9,5,77,48])
 
  # aggregate RDD elements using addition function
  sum = numbers.reduce(lambda a,b:a+b)
 
  print ("sum is : " + str(sum))
'''

f = open('testsparkjob.py', 'w')
f.write(sparkjob)
f.close()

In [18]:
!/content/spark-3.1.1-bin-hadoop2.7/bin/spark-submit /content/testsparkjob.py

21/05/26 11:48:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
21/05/26 11:48:15 INFO SparkContext: Running Spark version 3.1.1
21/05/26 11:48:15 INFO ResourceUtils: ==============================================================
21/05/26 11:48:15 INFO ResourceUtils: No custom resources configured for spark.driver.
21/05/26 11:48:15 INFO ResourceUtils: ==============================================================
21/05/26 11:48:15 INFO SparkContext: Submitted application: Read Text to RDD - Python
21/05/26 11:48:15 INFO ResourceProfile: Default ResourceProfile created, executor resources: Map(cores -> name: cores, amount: 1, script: , vendor: , memory -> name: memory, amount: 1024, script: , vendor: , offHeap -> name: offHeap, amount: 0, script: , vendor: ), task resources: Map(cpus -> name: cpus, amount: 1.0)
21/05/26 11:48:16 